<a href="https://colab.research.google.com/github/sibat119/pytorch-intro/blob/main/logistic_regression_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>